In [147]:
import pandas

titanic = pandas.read_csv("kaggle_titanic/train.csv")
test_data = pandas.read_csv("kaggle_titanic/test.csv")

In [188]:
# preprocessing

import re
from sklearn import preprocessing

def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""

def get_age_group(age):
    if age < 15:
        return 0
    if age < 50:
        return 1
    return 2

scaler = preprocessing.MinMaxScaler()

titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())
titanic["Embarked"] = titanic["Embarked"].fillna("S")

# new features
titanic["FamilySize"] = titanic["SibSp"] + titanic["Parch"]
titanic["NameLength"] = titanic["Name"].apply(lambda x: len(x))
titanic["Title"] = titanic["Name"].apply(get_title)


columns_labels = preprocessing.LabelEncoder()
train_columns_to_encode = ['Sex', 'Embarked', 'Title']

for col in train_columns_to_encode:
    titanic[col] = columns_labels.fit_transform(titanic[col])
    
titanic["Age"] = scaler.fit_transform(titanic["Age"])
# titanic["Embarked"] = scaler.fit_transform(titanic["Embarked"].astype(float))
# titanic["Parch"] = scaler.fit_transform(titanic["Parch"].astype(float))
# titanic["Pclass"] = scaler.fit_transform(titanic["Pclass"].astype(float))
    

In [189]:
# features exctractin

from sklearn import cross_validation
from sklearn.feature_selection import SelectKBest, f_classif

predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Embarked", "Title"]

# selector = SelectKBest(f_classif, k=5)
# selector.fit(titanic[predictors], titanic["Survived"])


In [190]:
def class_to_vect(classification):
    cls_count = 2
    vect = []
    for cls in classification:
        v = [0] * cls_count
        v[cls] = 1
        vect.append(v)
    return np.array(vect)


Two outputs approach:

In [211]:
# classification and cross validation

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
import numpy as np
from matplotlib import pyplot as plt
from keras.models import Sequential
from sklearn.cross_validation import KFold
from keras.layers.core import Dense, Dropout, Activation

# alg = LogisticRegression(random_state=1)
# alg = GradientBoostingClassifier()
nn = Sequential()
nn.add(Dense(len(predictors), output_dim=5, activation='tanh'))
nn.add(Dense(5, output_dim=5, activation='tanh'))
nn.add(Dropout(0.1))
nn.add(Dense(5, output_dim=2, activation='softmax'))
nn.compile(loss='binary_crossentropy', optimizer='Adam')

kf = KFold(titanic.shape[0], n_folds=10, random_state=1)
predictions = []
for train, test in kf:
    train_predictors = titanic[predictors].as_matrix()[train,:]
    target = class_to_vect(titanic.Survived.as_matrix())
    target_train = class_to_vect(titanic.Survived.iloc[train].as_matrix())
    nn.fit(train_predictors, target[train], nb_epoch=50, show_accuracy=True)
    test_predictions = nn.predict_classes(titanic[predictors].as_matrix()[test,:])
    predictions.append(test_predictions)
predictions = np.concatenate(predictions)


# for i in range(10, 200, 10):
#     alg = AdaBoostClassifier(n_estimators=80)
#     alg = KNeighborsClassifier(n_neighbors=i)
#     alg = RandomForestClassifier(random_state=0, n_estimators=i, min_samples_split=4, min_samples_leaf=2)
# scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic["Survived"], cv=3)
# score_means.append(scores.mean())
# # plt.plot(score_means)
# print(score_means)


Epoch 0
801/801 [==============================] - 0s - loss: 0.8678 - acc: 0.6205     
Epoch 1
801/801 [==============================] - 0s - loss: 0.8382 - acc: 0.6217     
Epoch 2
801/801 [==============================] - 0s - loss: 0.8052 - acc: 0.6192     
Epoch 3
801/801 [==============================] - 0s - loss: 0.7822 - acc: 0.6192     
Epoch 4
801/801 [==============================] - 0s - loss: 0.7564 - acc: 0.6105     
Epoch 5
801/801 [==============================] - 0s - loss: 0.7444 - acc: 0.6030     
Epoch 6
801/801 [==============================] - 0s - loss: 0.7155 - acc: 0.6067     
Epoch 7
801/801 [==============================] - 0s - loss: 0.7068 - acc: 0.5968     
Epoch 8
801/801 [==============================] - 0s - loss: 0.7071 - acc: 0.5868     
Epoch 9
801/801 [==============================] - 0s - loss: 0.7104 - acc: 0.5830     
Epoch 10
801/801 [==============================] - 0s - loss: 0.6970 - acc: 0.6030     
Epoch 11
801/801 [=============

Evaluation

In [210]:
def perc_score(actual, prediction):
    error = 0
    N = len(predictions)
    for i in range(N):
        if predictions[i] == actual.iloc[i]:
            error += 1
    return float(error) / N

print(perc_score(titanic.Survived, predictions))

0.7845117845117845


In [157]:
# processing test data 

test_data["Age"] = test_data["Age"].fillna(titanic["Age"].median())
test_data["Embarked"] = test_data["Embarked"].fillna("S")
test_data["Fare"] = test_data["Fare"].fillna(titanic["Fare"].median())

# new features
test_data["FamilySize"] = test_data["SibSp"] + test_data["Parch"]
test_data["NameLength"] = test_data["Name"].apply(lambda x: len(x))
test_data["Title"] = test_data["Name"].apply(get_title)

for col in train_columns_to_encode:
    test_data[col] = columns_labels.fit_transform(test_data[col])


In [141]:
# predict test data

# alg.fit(titanic[predictors], titanic["Survived"])
nn.fit(titanic[predictors].as_matrix(), class_to_vect(titanic.Survived.as_matrix()), nb_epoch=30, show_accuracy=True, validation_split=0.1)
predictions = nn.predict_classes(test_data[predictors].as_matrix())

submission = pandas.DataFrame({
        "PassengerId": test_data["PassengerId"],
        "Survived": predictions
    })
submission.to_csv("kaggle_titanic/submission.csv", index=False)

Train on 801 samples, validate on 90 samples
Epoch 0
801/801 [==============================] - 0s - loss: 0.2416 - acc: 0.6155 - val_loss: 0.2411 - val_acc: 0.6222
Epoch 1
801/801 [==============================] - 0s - loss: 0.2415 - acc: 0.6155 - val_loss: 0.2409 - val_acc: 0.6222
Epoch 2
801/801 [==============================] - 0s - loss: 0.2413 - acc: 0.6155 - val_loss: 0.2407 - val_acc: 0.6222
Epoch 3
801/801 [==============================] - 0s - loss: 0.2411 - acc: 0.6155 - val_loss: 0.2404 - val_acc: 0.6222
Epoch 4
801/801 [==============================] - 0s - loss: 0.2409 - acc: 0.6155 - val_loss: 0.2402 - val_acc: 0.6222
Epoch 5
801/801 [==============================] - 0s - loss: 0.2407 - acc: 0.6155 - val_loss: 0.2400 - val_acc: 0.6222
Epoch 6
801/801 [==============================] - 0s - loss: 0.2405 - acc: 0.6155 - val_loss: 0.2398 - val_acc: 0.6222
Epoch 7
801/801 [==============================] - 0s - loss: 0.2403 - acc: 0.6155 - val_loss: 0.2396 - val_acc: 0.